# This initial script is written by salma to create virus term variants out of curated dictionaries. This was further improved

### Pointers:
* Adding '2019', '2019novel', '2019new', '2019 novel', '2019 new' as prefixes or 2019 as suffix (only if '19' was not in the virus name)
* interchanging corona virus and coronavirus
* removing virus names containing the same word twice or containing both 'new' and 'novel'
* interchange "wuhan" and "hubei"
 
Also lowercase terms and remove hyphens

In [1]:
import pandas as pd
import re

In [2]:
infile_virus = "../data/processed_terms/ncbi_termsmerged_virus_sars_grm_lex.xlsx"
df_virus = pd.read_excel(infile_virus)
df_virus

,TERM,INCLUDE
0,new severe acute respiratory syndrome coronavi...,y
1,China COV,y
2,coronaviruses,y
3,severe acute respiratory syndrome coronavirus ...,y
4,SARS-CoV-2 infected COV,n
...,...,...
161,2019-novel coronavirus (2019-nCoV) related cor...,n
162,SARS-CoV-2 (Severe acute respiratory syndrome ...,y
163,Novel Coronavirus causing COVID-19,y
164,COVID-19 cov,y


In [3]:
df_virus = df_virus[df_virus["INCLUDE"]=="y"]
df_virus

,TERM,INCLUDE
0,new severe acute respiratory syndrome coronavi...,y
1,China COV,y
2,coronaviruses,y
3,severe acute respiratory syndrome coronavirus ...,y
5,ncov,y
...,...,...
160,novel SARS-CoV-2 coronavirus,y
162,SARS-CoV-2 (Severe acute respiratory syndrome ...,y
163,Novel Coronavirus causing COVID-19,y
164,COVID-19 cov,y


In [4]:
virus_terms = df_virus["TERM"].tolist()
virus_terms

['new severe acute respiratory syndrome coronavirus 2',
 'China COV',
 'coronaviruses',
 'severe acute respiratory syndrome coronavirus (CoV)-2',
 'ncov',
 'new SARS-CoV-2 coronavirus',
 'COVID-19 viruses',
 'coronavirus responsible for COVID-19',
 'SARS (severe acute respiratory syndrome)-CoV-2 virus',
 'novel coronavirus Syndrome-Coronavirus',
 'Wuhan COV',
 'Wuhan pneumonia virus',
 'SARS-CoV-2 (CoV-2) coronavirus',
 'new SARS-CoV-2 virus',
 'SARS-CoV2 (CoV2) virus',
 'virus causing 2019 coronavirus disease',
 'virus causing COVID-19',
 'novel coronavirus that causes COVID-19 disease',
 'SARS-CoV-2 coronaviruses',
 '2019 COV',
 'novel coronavirus 2019 (COVID-19) virus',
 'virus that causes coronavirus disease',
 'CoV2',
 'new coronavirus disease COV',
 'Chinese coronavirus',
 'novel severe acute respiratory syndrome (SARS)-coronavirus (CoV)2',
 'coronavirus that causes COVID-19',
 'virus that causes COVID-19 infection',
 'new coronavirus COV',
 'novel coronavirus responsible for COV

In [5]:
def remove_hyphens_lowercase(list_):
    new_set = set()
    for term in list_:
        new_set.update([term.lower().replace("-"," ")])
    return sorted(list(new_set))


def _2019_var(list_):
    '''
    create variants with 2019 and new/novel
    '''
    variants_prefix = ['2019novel', '2019new', '2019 novel', '2019 new']
    
    new_set = set()
    
    for term in list_:
        new_set.update([term])
        if "2019" not in term and "19" not in term:
            # add 2019 as prefix and suffix
            new_set.update([term + " 2019"])
            new_set.update(["2019 " + term])            
            
            for var in variants_prefix:
                new_set.update([var + " " + term])

        
    new_set_2 = set()
    #check for duplicates
    for term in new_set:
        if check_duplicate_words_from_single_term(term):
            continue
        elif check_if_both_new_novel(term):
            continue
        elif re.compile(r'\b2019new\b').search(term) and re.compile(r'\bnew\b').search(term):
            continue
        elif re.compile(r'\b2019novel\b').search(term) and re.compile(r'\bnovel\b').search(term):
            continue
        else:
            new_set_2.update([term])
        
    return sorted(new_set_2)
            
    
def check_duplicate_words_from_single_term(term):
    words = term.split()
    return len(words) > len(set(words))

def check_if_both_new_novel(term):
    return "new" in term and "novel" in term

def remove_duplicates(list_):
    '''
    remove duplicate terms and remove "new"/"novel" multiple occurances
    '''
    new_set = set()
    
    for term in list_:
        if check_duplicate_words_from_single_term(term):
            print("duplicate: ", term)
            continue
        elif check_if_both_new_novel(term):
            print("new/novel: ",term)
            continue
        else:
            new_set.update([term])
    
    return sorted(new_set) 
            
def interchance_wuhan_hubei(list_):
    '''
    interchanging wuhan and hubei
    '''
    new_set = set()
    
    for term in list_:
        term = term.lower()
        
        new_set.update([term])
        
        if "wuhan" in term:
            new_set.update([term.replace("wuhan", "hubei")])
        elif "hubei" in term:
            new_set.update([term.replace("hubei", "wuhan")])
            
    return sorted(new_set)

def interchange_corona(list_):
    '''
    interchange corona and coronavirus terms 
    '''
    new_set = set()
    for term in list_:
        term = term.lower()
        new_set.update([term])
        if "corona virus" in term:
            continue
        elif "coronavirus" in term:
            if not "acute respiratory syndrome coronavirus" in term:
                new_set.update([term.replace("coronavirus", "corona")])
        else:
            new_set.update([re.sub("\bcorona\b", "coronavirus", term)])
            
    return sorted(new_set)

In [6]:
list_lowercase = remove_hyphens_lowercase(virus_terms)
list_lowercase

['19 coronavirus',
 '19 cov',
 '19 ncov',
 '2019 cov',
 '2019 ncov virus',
 '2019 ncov, 2019 novel coronavirus',
 '2019 new coronavirus',
 '2019 novel coronavirus disease cov',
 'acute respiratory syndrome coronavirus 2',
 'acute respiratory syndrome coronavirus causing coronavirus disease 2019',
 'china cov',
 'china virus',
 'chinese coronavirus',
 'coronavirus',
 'coronavirus (sars cov 2) causing covid 19',
 'coronavirus (sars cov 2) responsible for coronavirus disease',
 'coronavirus causing coronavirus disease 2019',
 'coronavirus disease 2019 (sars cov 2) virus',
 'coronavirus disease 2019 cov',
 'coronavirus disease cov',
 'coronavirus infectious disease cov',
 'coronavirus responsible for coronavirus disease 2019',
 'coronavirus responsible for covid 19',
 'coronavirus that causes coronavirus disease 2019',
 'coronavirus that causes covid 19',
 'coronaviruses',
 'coronaviruses that causes coronavirus disease 2019',
 'cov 19 virus',
 'cov 2 virus',
 'cov2',
 'cov2 coronavirus',


In [7]:
list_lowercase_2019 = _2019_var(list_lowercase)
list_lowercase_2019

['19 coronavirus',
 '19 cov',
 '19 ncov',
 '2019 acute respiratory syndrome coronavirus 2',
 '2019 china cov',
 '2019 china virus',
 '2019 chinese coronavirus',
 '2019 coronavirus',
 '2019 coronavirus disease cov',
 '2019 coronavirus infectious disease cov',
 '2019 coronaviruses',
 '2019 cov',
 '2019 cov 2 virus',
 '2019 cov2',
 '2019 cov2 coronavirus',
 '2019 hcov 2',
 '2019 human coronavirus 2',
 '2019 human coronaviruses (hcovs) 2',
 '2019 ncov',
 '2019 ncov 2',
 '2019 ncov virus',
 '2019 new acute respiratory syndrome coronavirus 2',
 '2019 new china cov',
 '2019 new china virus',
 '2019 new chinese coronavirus',
 '2019 new coronavirus',
 '2019 new coronavirus cov',
 '2019 new coronavirus disease cov',
 '2019 new coronavirus infection cov',
 '2019 new coronavirus infections cov',
 '2019 new coronavirus infectious disease cov',
 '2019 new coronavirus pneumonia cov',
 '2019 new coronaviruses',
 '2019 new cov',
 '2019 new cov 2 virus',
 '2019 new cov2',
 '2019 new cov2 coronavirus',
 

In [8]:
list_lowercase_2019_corona = interchange_corona(list_lowercase_2019)
list_lowercase_2019_corona

['19 corona',
 '19 coronavirus',
 '19 cov',
 '19 ncov',
 '2019 acute respiratory syndrome coronavirus 2',
 '2019 china cov',
 '2019 china virus',
 '2019 chinese corona',
 '2019 chinese coronavirus',
 '2019 corona',
 '2019 corona disease cov',
 '2019 corona infectious disease cov',
 '2019 coronaes',
 '2019 coronavirus',
 '2019 coronavirus disease cov',
 '2019 coronavirus infectious disease cov',
 '2019 coronaviruses',
 '2019 cov',
 '2019 cov 2 virus',
 '2019 cov2',
 '2019 cov2 corona',
 '2019 cov2 coronavirus',
 '2019 hcov 2',
 '2019 human corona 2',
 '2019 human coronaes (hcovs) 2',
 '2019 human coronavirus 2',
 '2019 human coronaviruses (hcovs) 2',
 '2019 ncov',
 '2019 ncov 2',
 '2019 ncov virus',
 '2019 new acute respiratory syndrome coronavirus 2',
 '2019 new china cov',
 '2019 new china virus',
 '2019 new chinese corona',
 '2019 new chinese coronavirus',
 '2019 new corona',
 '2019 new corona cov',
 '2019 new corona disease cov',
 '2019 new corona infection cov',
 '2019 new corona i

In [9]:
list_lowercase_2019_corona_wuhan = interchance_wuhan_hubei(list_lowercase_2019_corona)
list_lowercase_2019_corona_wuhan

['19 corona',
 '19 coronavirus',
 '19 cov',
 '19 ncov',
 '2019 acute respiratory syndrome coronavirus 2',
 '2019 china cov',
 '2019 china virus',
 '2019 chinese corona',
 '2019 chinese coronavirus',
 '2019 corona',
 '2019 corona disease cov',
 '2019 corona infectious disease cov',
 '2019 coronaes',
 '2019 coronavirus',
 '2019 coronavirus disease cov',
 '2019 coronavirus infectious disease cov',
 '2019 coronaviruses',
 '2019 cov',
 '2019 cov 2 virus',
 '2019 cov2',
 '2019 cov2 corona',
 '2019 cov2 coronavirus',
 '2019 hcov 2',
 '2019 hubei cov',
 '2019 hubei novel corona',
 '2019 hubei novel coronavirus',
 '2019 hubei pneumonia virus',
 '2019 hubei seafood market pneumonia virus',
 '2019 hubei virus',
 '2019 human corona 2',
 '2019 human coronaes (hcovs) 2',
 '2019 human coronavirus 2',
 '2019 human coronaviruses (hcovs) 2',
 '2019 ncov',
 '2019 ncov 2',
 '2019 ncov virus',
 '2019 new acute respiratory syndrome coronavirus 2',
 '2019 new china cov',
 '2019 new china virus',
 '2019 new c

In [10]:
len(list_lowercase), len(list_lowercase_2019), len(list_lowercase_2019_corona), len(list_lowercase_2019_corona_wuhan)

(139, 401, 588, 622)

In [11]:
list_final = remove_duplicates(list_lowercase_2019_corona_wuhan)
list_final

['19 corona',
 '19 coronavirus',
 '19 cov',
 '19 ncov',
 '2019 acute respiratory syndrome coronavirus 2',
 '2019 china cov',
 '2019 china virus',
 '2019 chinese corona',
 '2019 chinese coronavirus',
 '2019 corona',
 '2019 corona disease cov',
 '2019 corona infectious disease cov',
 '2019 coronaes',
 '2019 coronavirus',
 '2019 coronavirus disease cov',
 '2019 coronavirus infectious disease cov',
 '2019 coronaviruses',
 '2019 cov',
 '2019 cov 2 virus',
 '2019 cov2',
 '2019 cov2 corona',
 '2019 cov2 coronavirus',
 '2019 hcov 2',
 '2019 hubei cov',
 '2019 hubei novel corona',
 '2019 hubei novel coronavirus',
 '2019 hubei pneumonia virus',
 '2019 hubei seafood market pneumonia virus',
 '2019 hubei virus',
 '2019 human corona 2',
 '2019 human coronaes (hcovs) 2',
 '2019 human coronavirus 2',
 '2019 human coronaviruses (hcovs) 2',
 '2019 ncov',
 '2019 ncov 2',
 '2019 ncov virus',
 '2019 new acute respiratory syndrome coronavirus 2',
 '2019 new china cov',
 '2019 new china virus',
 '2019 new c

In [12]:
len(list_lowercase), len(list_lowercase_2019), len(list_lowercase_2019_corona), len(list_lowercase_2019_corona_wuhan), len(list_final)

(139, 401, 588, 622, 622)

In [13]:
outfile = "../results/20220321_ra_additionally_processed_dictionaries/20220321_ra_virus_processed.txt"

with open(outfile, 'w') as f:
    for term in sorted(list_final):
        f.write(term + "\n")

## Merge files with old

In [18]:
orig_file = "../data/AITSLAB_dictionaries/Virus_SARS-CoV-2.txt"
orig_set = set()

with open (orig_file) as f:
    for line in f.readlines()[:-5]:
        line = line.strip()
        orig_set.update([line])
sorted(orig_set)

['2019 chinese corona virus',
 '2019 chinese coronavirus',
 '2019 corona virus',
 '2019 coronavirus',
 '2019 human corona virus',
 '2019 human coronavirus',
 '2019 ncov',
 '2019 new chinese corona virus',
 '2019 new chinese coronavirus',
 '2019 new corona virus',
 '2019 new coronavirus',
 '2019 new sars corona virus 2',
 '2019 new sars corona virus2',
 '2019 new sars coronavirus 2',
 '2019 new sars coronavirus2',
 '2019 new sars cov 2',
 '2019 new sarscov2',
 '2019 new seafood market corona virus',
 '2019 new seafood market coronavirus',
 '2019 new seafood market virus',
 '2019 new severe acute respiratory syndrome corona virus 2',
 '2019 new severe acute respiratory syndrome corona virus2',
 '2019 new severe acute respiratory syndrome coronavirus 2',
 '2019 new severe acute respiratory syndrome coronavirus2',
 '2019 new severe acute respiratory syndromerelated corona virus 2',
 '2019 new severe acute respiratory syndromerelated corona virus2',
 '2019 new severe acute respiratory syndr

In [19]:
len(list_final), len(orig_set)

(622, 218)

In [20]:
merged_set = set.union(set(list_final),orig_set)
len(merged_set)

802

In [21]:
merged_set

{'19 corona',
 '19 coronavirus',
 '19 cov',
 '19 ncov',
 '2019 acute respiratory syndrome coronavirus 2',
 '2019 china cov',
 '2019 china virus',
 '2019 chinese corona',
 '2019 chinese corona virus',
 '2019 chinese coronavirus',
 '2019 corona',
 '2019 corona disease cov',
 '2019 corona infectious disease cov',
 '2019 corona virus',
 '2019 coronaes',
 '2019 coronavirus',
 '2019 coronavirus disease cov',
 '2019 coronavirus infectious disease cov',
 '2019 coronaviruses',
 '2019 cov',
 '2019 cov 2 virus',
 '2019 cov2',
 '2019 cov2 corona',
 '2019 cov2 coronavirus',
 '2019 hcov 2',
 '2019 hubei cov',
 '2019 hubei novel corona',
 '2019 hubei novel coronavirus',
 '2019 hubei pneumonia virus',
 '2019 hubei seafood market pneumonia virus',
 '2019 hubei virus',
 '2019 human corona 2',
 '2019 human corona virus',
 '2019 human coronaes (hcovs) 2',
 '2019 human coronavirus',
 '2019 human coronavirus 2',
 '2019 human coronaviruses (hcovs) 2',
 '2019 ncov',
 '2019 ncov 2',
 '2019 ncov virus',
 '2019 

In [22]:
outfile_merged = "../results/20220321_ra_additionally_processed_dictionaries/20220321_ra_virus_merged.txt"
with open(outfile_merged, 'w') as f:
    for term in sorted(merged_set):
        f.write(term + "\n")